## Loading the environment

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# MODEL = "gpt-3.5-turbo"
MODEL = "llama3.1"

# Sam Altman: OpenAI CEO on GPT-4, ChatGPT, and the Future of AI
YOUTUBE_VIDEO_URL = "https://www.youtube.com/watch?v=L_Guz73e6fw"


## Setting up llm/slm model with its corresponding embeddings

In [2]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

if MODEL.startswith("gpt"):
    model = ChatOpenAI(api_key=OPENAI_API_KEY,model=MODEL)
    embeddings = OpenAIEmbeddings()
else:
    model = Ollama(model=MODEL)
    embeddings = OllamaEmbeddings(model=MODEL)

Simple model test to verify if it's working.

Extracting the response by chaining the model with an output parser

In [3]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
chain = model | parser
chain.invoke("Tell me a joke")

"Here's one:\n\nWhat do you call a fake noodle?\n\nAn impasta."

## Setting up the prompt templates

In [4]:
from langchain.prompts import ChatPromptTemplate

template = """
Answer the question based on the context below.
If you can't answer the question, reply "I don't know"

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))

Human: 
Answer the question based on the context below.
If you can't answer the question, reply "I don't know"

Context: Here is some context

Question: Here is a question



Simple test to see if the model & prompt template was done correctly

In [5]:
chain = prompt | model | parser
chain.invoke({
    "context": "The name I was given was Daniel",
    "question": "What's my name ?"
})

'Daniel.'

## Alternate translation prompt

In [ ]:
from operator import itemgetter

translation_prompt = ChatPromptTemplate.from_template(
    "Translate {answer} to {language}"
)

translation_chain = (
    {"answer": chain, "language": itemgetter("language")} | translation_prompt | model | parser
)

translation_chain.invoke(
    {
        "context": "Daniel want to order calamari for lunch. He does'nt like anything else.",
        "question": "How many dishes did Daniel ordered?",
        "language": "français",
    }
)

## Transcribing YouTube Video

Transcribing a ~2 and half hour video, the following code might take a few minutes.

Note : Using pytubefix instead of pytube. The developers of this library are trying to fix the HTTP 400 response error as of August 21st 2024...

In [6]:
import tempfile
from pytubefix import YouTube
import whisper


if not os.path.exists("ytb_transcript.txt"):
    youtube = YouTube(YOUTUBE_VIDEO_URL)
    audio = youtube.streams.filter(only_audio=True).first()

    # "base" model used for ok accuracy and fast transcribing
    whisper_model = whisper.load_model("base")

    with tempfile.TemporaryDirectory() as tmpdir:
        file = audio.download(output_path=tmpdir)
        transcription = whisper_model.transcribe(file, fp16=False)["text"].strip()

        with open("ytb_transcript.txt", "w") as file:
            file.write(transcription)

## Splitting the transcript into smaller documents

In [7]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = TextLoader("ytb_transcript.txt")
txt_documents = loader.load()
txt_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
documents = txt_splitter.split_documents(txt_documents)

## Loading the documents into a vector store 

In [8]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(documents, embeddings)

/Users/danielgiaoo/Developer/GitHub/rag-youtube-qna-app/.venv/lib/python3.12/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


## Retrieve & passing a setup map through the chain

In [9]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

retriever = vectorstore.as_retriever()
setup = RunnableParallel(context=retriever, question=RunnablePassthrough())
chain = setup | prompt | model | parser

Lets try it out on our newly created chain!

In [10]:
chain.invoke("What is the future of AI ?")

'It seems like the conversation doesn\'t explicitly state what the future of AI will be, but rather hints at its potential and the opinions of some experts in the field.\n\nThe quote from Alan Turing (1951) mentions that "it seems probable that once the machine thinking method has started, it would not take long to outstrip our feeble powers. At some stage, therefore, we should have to expect the machines to take control."\n\nHowever, the rest of the conversation doesn\'t provide a clear answer on what this future will hold.\n\nI don\'t know'